In [6]:
from sqlalchemy import create_engine, Column, Integer
from sqlalchemy.orm import declarative_base, Session

Base = declarative_base()

# Simple table for demo
class Counter(Base):
    __tablename__ = "counters"
    id = Column(Integer, primary_key=True)
    value = Column(Integer)
    version = Column(Integer, nullable=False, default=1)

    __mapper_args__ = {
        "version_id_col": version
    }

# Set up in-memory SQLite and create the table
engine = create_engine("sqlite:///:memory:", echo=False)
Base.metadata.create_all(engine)

# Insert initial value
with Session(engine) as session:
    session.add(Counter(id=1, value=0))
    session.commit()

# Simulate two separate sessions modifying the same row
with Session(engine) as s1, Session(engine) as s2:
    c1 = s1.get(Counter, 1)
    c2 = s2.get(Counter, 1)

    # Session 1 modifies and commits
    c1.value += 1
    s1.commit()

    # Session 2 modifies the old value (stale copy)
    c2.value += 10
    try:
        s2.commit()
        result = "No error raised. Final value in DB = " + str(s2.get(Counter, 1).value)
    except Exception as e:
        result = f"Conflict detected: {e}"

result


"Conflict detected: UPDATE statement on table 'counters' expected to update 1 row(s); 0 were matched."

1+10=11 이어야 함. session가 덮어 써버림.

In [ ]:
from sqlalchemy import create_engine, Column, Integer
from sqlalchemy.orm import declarative_base, Session

Base = declarative_base()

# Simple table for demo
class Counter(Base):
    __tablename__ = "counters"
    id = Column(Integer, primary_key=True)
    value = Column(Integer)
    version = Column(Integer, nullable=False, default=1)

    __mapper_args__ = {
        "version_id_col": version
    }

# Set up in-memory SQLite and create the table
engine = create_engine("sqlite:///:memory:", echo=False)
Base.metadata.create_all(engine)

# Insert initial value
with Session(engine) as session:
    session.add(Counter(id=1, value=0))
    session.commit()

# Simulate two separate sessions modifying the same row
with Session(engine) as s1, Session(engine) as s2:
    c1 = s1.get(Counter, 1)

    # Session 1 modifies and commits
    c1.value += 1
    s1.commit()

    # Session 2 modifies the old value (stale copy)
    c2 = s2.get(Counter, 1)
    c2.value += 10
    try:
        s2.commit()
        result = "No error raised. Final value in DB = " + str(s2.get(Counter, 1).value)
    except Exception as e:
        result = f"Conflict detected: {e}"

result

'No error raised. Final value in DB = 11'

In [38]:
ALL_SESSIONS = []

def create_session():
    session = Session(engine)
    ALL_SESSIONS.append(session)
    return session


In [39]:
session1 = create_session()

In [40]:
session1